In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import csv
import re
import os
from urllib.error import HTTPError
from astropy.time import Time
from astropy.coordinates import SkyCoord
import astropy.units as u


In [5]:
# Move up one directory from ia_supernovae_stuff to Query_VLASS_2
%cd ..

# Now you should be in Query_VLASS_2, you can import the module
import vlass_search as vs

# After importing, if you want to go back to your working folder:
%cd ia_supernovae_stuff

d:\summer_research_code\Query_VLASS_2
File downloaded to: VLASS_dyn_summary.php
d:\summer_research_code\Query_VLASS_2\ia_supernovae_stuff


### Function to Get Unique Part of Fits File

In [8]:
import os
from urllib.request import urlopen
from contextlib import closing
import numpy as np
from astropy.coordinates import SkyCoord

def get_unique_fits_part(name, epoch, c):
    """
    Retrieves the unique part of the FITS file download link for a given source name, epoch, and coordinates.

    Parameters
    ----------
    name : str
        The name of the source.
    epoch : str
        The epoch of the observation (e.g., 'VLASS3.2').
    coordinates : SkyCoord
        The coordinates as a SkyCoord object.

    Returns
    -------
    str
        The unique part of the FITS download link or a message indicating failure.
    """
    tiles = vs.get_tiles()
    tilenames, epochs, obsdates = vs.search_tiles(tiles, c)
    
    # Assume that it is the same tile name regardless of epoch 
    tile = tilenames[0]

    subtiles, c_tiles = vs.get_subtiles(tile, epoch)
    print("Tile Found:")
    print(tile, epoch)
    
    # Find angular separation from the tiles to the location
    dist = c.separation(c_tiles)
    # Find tile with the smallest separation 
    subtile = subtiles[np.argmin(dist)]
    
    # Unique part of the link (e.g., tile and file name)
    unique_part = "%s/%s.I.iter1.image.pbcor.tt0.subim.fits" % (tile, subtile[:-1])
    
    return unique_part  # Return the unique part of the link



In [7]:
from astropy.coordinates import SkyCoord
from astropy import units as u

# Example coordinates
coord_str = "10:35:32.09 +37:38:59.0"
c = SkyCoord(coord_str, unit=(u.hourangle, u.deg))

# Example transient name and epoch
name = "ZTF17aabtvsy"
epoch = "VLASS2.2"

get_unique_fits_part(name, epoch, c)

https://archive-new.nrao.edu/vlass/quicklook/VLASS2.2/T20t14/
Tile Found:
T20t14 VLASS2.2


'T20t14/VLASS2.2.ql.T20t14.J103631+373000.10.2048.v1.I.iter1.image.pbcor.tt0.subim.fits'

In [9]:
df = pd.read_csv("final_ia_transients.csv")

In [11]:
import pandas as pd

def add_ra_dec_to_transients(final_csv, supernovae_csv, output_csv):
    # Load the CSV files
    transients_df = pd.read_csv(final_csv)
    supernovae_df = pd.read_csv(supernovae_csv)

    # Select only 'Transient Name', 'RA', and 'Dec' from the supernovae DataFrame
    supernovae_df = supernovae_df[['Transient Name', 'RA', 'Dec']]

    # Merge the DataFrames on 'Transient Name'
    merged_df = pd.merge(transients_df, supernovae_df, on='Transient Name', how='left')

    # Save the updated DataFrame to a new CSV file
    merged_df.to_csv(output_csv, index=False)
    print(f"RA and Dec columns added to {output_csv}")

# Example usage
add_ra_dec_to_transients('final_ia_transients.csv', 'ia_supernovae.csv', 'final_transients_with_coords.csv')


KeyError: "['Transient Name'] not in index"